In [1]:
## load useful libraries
suppressWarnings(suppressPackageStartupMessages(library(rhdf5)))
suppressWarnings(suppressPackageStartupMessages(library(qvalue)))
suppressWarnings(suppressPackageStartupMessages(library(dplyr)))
suppressWarnings(suppressPackageStartupMessages(library(cowplot)))
suppressWarnings(suppressPackageStartupMessages(library(tidyverse)))
suppressWarnings(suppressPackageStartupMessages(library(ggbeeswarm)))
suppressWarnings(suppressPackageStartupMessages(library(ggthemes)))
suppressWarnings(suppressPackageStartupMessages(library(scater)))
# suppressWarnings(suppressPackageStartupMessages(library(SingleCellExperiment)))
suppressWarnings(suppressPackageStartupMessages(library(edgeR)))
suppressWarnings(suppressPackageStartupMessages(library(limma)))
suppressWarnings(suppressPackageStartupMessages(library(ggplot2)))
suppressWarnings(suppressPackageStartupMessages(library(ISLR)))
suppressWarnings(suppressPackageStartupMessages(library(GGally)))
suppressWarnings(suppressPackageStartupMessages(library(scran)))
suppressWarnings(suppressPackageStartupMessages(library(ComplexHeatmap)))
suppressWarnings(suppressPackageStartupMessages(library(circlize)))

In [2]:
step.pct = 0.025
window.pct = 0.25
output.dir = "/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_processed/scQTLs/binning_analysis/input_files_noday/"
sce.file = "/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_processed/merged/20180618/sce_merged_afterqc_filt_allexpts.rds"

In [5]:
sce = readRDS(sce.file)
sce

class: SingleCellExperiment 
dim: 11231 36044 
metadata(4): log.exprs.offset log.exprs.offset log.exprs.offset
  log.exprs.offset
assays(3): tpm counts logcounts
rownames(11231): ENSG00000000003_TSPAN6 ENSG00000000419_DPM1 ...
  ENSG00000272325_NUDT3 ENSG00000272398_CD24
rowData names(17): exprs_collapsed_to ensembl_transcript_id ...
  is_intop2000hvg is_hvg
colnames(36044): 21843_1#10 21843_1#100 ... 24539_8#97 24539_8#98
colData names(123): adj_x adj_y ... princ_curve princ_curve_scaled01
reducedDimNames(1): PCA
spikeNames(1): ERCC

In [6]:
tot = ncol(sce)
window = window.pct*tot
step = step.pct*tot

In [7]:
print (tot)
print (window)
print(step)

[1] 36044
[1] 9011
[1] 901.1


In [8]:
get_combinations_nodays <- function(sceset){

    mysce = sceset
    mat = table(mysce$donor_short_id, mysce$experiment) 
    ord = which(table(mysce$donor_short_id, mysce$experiment) > 0, arr.ind = T)
    l = c()
    for (i in 1:dim(ord)[1]){
        l[i] <- paste0(rownames(mat)[ord[i,1]],"-",colnames(mat)[ord[i,2]])
    }
    l
}

In [9]:
get_means_byexpt_nodays <- function(sceset, ngenes, l){
    n = length(l)
    means <- matrix(0,nrow = ngenes, ncol = n)
    for (i in 1:n){
        don = gsub("*-.*","",l[i])
        expt = gsub(".*-","",l[i])
        sc <- logcounts(sceset)[,sceset$donor_short_id == don & sceset$experiment == expt]
        # print(length(sc))
        if (length(sc) == nrow(sceset)) {means[,i] <- sc}
        else {means[,i] <- rowMeans(sc)}
    }
    rownames(means) = rownames(sceset)
    colnames(means) = l
    means
}

In [24]:
make_sample_mapping_nodays <- function(pheno, sce){
    df <- data.frame(donor.expt = colnames(pheno))
#     print(head(df))
    df$donor = gsub("*-.*","",df$donor.expt)
    df$expt = gsub(".*-","",df$donor.expt)
#     print(head(df))
    df1 <- data.frame(donor_long_id = unique(sce$donor_long_id), donor = unique(sce$donor_short_id))
#     print(head(df1))
    df2 <- left_join(df, df1, by = "donor")
#     print(head(df2))
    # genotype_samples phenotype_samples
    samples = df2[,c(4,1)]
#     print(head(samples))
    colnames(samples) = c()
    samples
}

In [25]:
#### binning by pseudotime
df = data.frame(cells = colnames(sce), pseudo = sce$pseudo)
df = df[order(df$pseudo),]
df$rank = 1:nrow(df)
head(df)

,cells,pseudo,rank
25271_7#119,25271_7#119,0.000000000,1
24842_6#234,24842_6#234,0.005807972,2
25271_8#231,25271_8#231,0.008059015,3
25271_7#264,25271_7#264,0.010437261,4
24229_3#255,24229_3#255,0.010756032,5
25271_7#191,25271_7#191,0.010763582,6


In [26]:
ii
bin_cells = df[df$rank >= round(ii) & df$rank < round(ii+window),"cells"]
mysce = sce[,bin_cells]
mysce

[1] 1

class: SingleCellExperiment 
dim: 11231 9011 
metadata(4): log.exprs.offset log.exprs.offset log.exprs.offset
  log.exprs.offset
assays(3): tpm counts logcounts
rownames(11231): ENSG00000000003_TSPAN6 ENSG00000000419_DPM1 ...
  ENSG00000272325_NUDT3 ENSG00000272398_CD24
rowData names(17): exprs_collapsed_to ensembl_transcript_id ...
  is_intop2000hvg is_hvg
colnames(9011): 25271_7#119 24842_6#234 ... 24086_6#127 24327_5#275
colData names(123): adj_x adj_y ... princ_curve princ_curve_scaled01
reducedDimNames(1): PCA
spikeNames(1): ERCC

In [27]:
length(seq(1, tot-window, by = step))
tot
seq(1, tot-window, by = step)

tot-window

j = 0
# print (j)
# print(head(seq(1, tot-window, by = step)))
for (ii in seq(1, tot-window, by = step)){
    print(ii)
#     if(ii < 234230){next}
    bin_cells = df[df$rank >= round(ii) & df$rank < round(ii+window),"cells"]
    mysce = sce[,bin_cells]
#     print(ncol(mysce[,mysce$day == "day0"]))
    l = get_combinations_nodays(mysce)
    m = get_means_byexpt_nodays(mysce, nrow(mysce), l)
    mean_pcs = prcomp(t(m))$x[,1:10]
    smf = make_sample_mapping_nodays(m, mysce)
    j = j + 1
    phenotype_filename = paste0(output.dir,"bin",j,".pheno")
    covariate_filename = paste0(output.dir,"bin",j,".covs")
    samplemapping_filename = paste0(output.dir,"bin",j,".samples")
    write.table(m, phenotype_filename, sep = "\t", quote = F, col.names = NA)
    write.table(mean_pcs, covariate_filename, sep = "\t", quote = F, col.names = NA)
    write.table(smf, samplemapping_filename, sep = "\t", quote = F, row.names = F, col.names = F)
}

[1] 30

[1] 36044

[1]     1.0   902.1  1803.2  2704.3  3605.4  4506.5  5407.6  6308.7  7209.8
[10]  8110.9  9012.0  9913.1 10814.2 11715.3 12616.4 13517.5 14418.6 15319.7
[19] 16220.8 17121.9 18023.0 18924.1 19825.2 20726.3 21627.4 22528.5 23429.6
[28] 24330.7 25231.8 26132.9

[1] 27033

[1] 1
[1] 902.1
[1] 1803.2
[1] 2704.3
[1] 3605.4
[1] 4506.5
[1] 5407.6
[1] 6308.7
[1] 7209.8
[1] 8110.9
[1] 9012
[1] 9913.1
[1] 10814.2
[1] 11715.3
[1] 12616.4
[1] 13517.5
[1] 14418.6
[1] 15319.7
[1] 16220.8
[1] 17121.9
[1] 18023
[1] 18924.1
[1] 19825.2
[1] 20726.3
[1] 21627.4
[1] 22528.5
[1] 23429.6
[1] 24330.7
[1] 25231.8
[1] 26132.9


In [29]:
j = 31
ii = 26132.9
bin_cells = df[df$rank >= round(ii),"cells"]
mysce = sce[,bin_cells]
#     print(ncol(mysce[,mysce$day == "day0"]))
l = get_combinations_nodays(mysce)
m = get_means_byexpt_nodays(mysce, nrow(mysce), l)
mean_pcs = prcomp(t(m))$x[,1:10]
smf = make_sample_mapping_nodays(m, mysce)

phenotype_filename = paste0(output.dir,"bin",j,".pheno")
covariate_filename = paste0(output.dir,"bin",j,".covs")
samplemapping_filename = paste0(output.dir,"bin",j,".samples")
write.table(m, phenotype_filename, sep = "\t", quote = F, col.names = NA)
write.table(mean_pcs, covariate_filename, sep = "\t", quote = F, col.names = NA)
write.table(smf, samplemapping_filename, sep = "\t", quote = F, row.names = F, col.names = F)

In [16]:
get_combinations <- function(sceset){
    # day0
    mysce = sceset[,sceset$day == "day0"]
    if (ncol(mysce) == 0){l0 <- c()}
    else {
        mat = table(mysce$donor_short_id, mysce$experiment) 
        ord = which(table(mysce$donor_short_id, mysce$experiment) > 0, arr.ind = T)
#         print (ord)
        l = c()
        for (i in 1:dim(ord)[1]){l[i] <- paste0(rownames(mat)[ord[i,1]],"-",colnames(mat)[ord[i,2]])}
        l0 = paste0(l,"-day0")
    }
    # day1
    mysce = sceset[,sceset$day == "day1"]
    if (ncol(mysce) == 0){l1 <- c()}
    else {
        mat = table(mysce$donor_short_id, mysce$experiment) 
        ord = which(table(mysce$donor_short_id, mysce$experiment) > 0, arr.ind = T)
#         print (ord)
        l = c()
        for (i in 1:dim(ord)[1]){l[i] <- paste0(rownames(mat)[ord[i,1]],"-",colnames(mat)[ord[i,2]])}
        l1 = paste0(l,"-day1")
    }
    # day2
    mysce = sceset[,sceset$day == "day2"]
    if (ncol(mysce) == 0){l2 <- c()}
    else {
        mat = table(mysce$donor_short_id, mysce$experiment) 
        ord = which(table(mysce$donor_short_id, mysce$experiment) > 0, arr.ind = T)
#         print (ord)
        l = c()
        for (i in 1:dim(ord)[1]){l[i] <- paste0(rownames(mat)[ord[i,1]],"-",colnames(mat)[ord[i,2]])}
        l2 = paste0(l,"-day2")
    }
    # day3
    mysce = sceset[,sceset$day == "day3"]
    if (ncol(mysce) == 0){l3 <- c()}
    else {
        mat = table(mysce$donor_short_id, mysce$experiment) 
        ord = which(table(mysce$donor_short_id, mysce$experiment) > 0, arr.ind = T)
#         print (ord)
        l = c()
        for (i in 1:dim(ord)[1]){l[i] <- paste0(rownames(mat)[ord[i,1]],"-",colnames(mat)[ord[i,2]])}
        l3 = paste0(l,"-day3")
    }
    # combine (aggregation by donor, experiment, day)
    l = c(l0, l1, l2, l3)
    l
}

In [17]:
get_means_byexpt <- function(sceset, ngenes, l){
    n = length(l)
    means <- matrix(0,nrow = ngenes, ncol = n)
    for (i in 1:n){
        don = gsub("*-.*","",l[i])
        expt = gsub("*-.*","",gsub(".*-e","e",l[i]))
        day = gsub(".*-","",l[i])
        sc <- logcounts(sceset)[,sceset$donor_short_id == don & sceset$experiment == expt & sceset$day == day]
        # print(length(sc))
        if (length(sc) == nrow(sceset)) {means[,i] <- sc}
        else {means[,i] <- rowMeans(sc)}
    }
    rownames(means) = rownames(sceset)
    colnames(means) = l
    means
}

In [18]:
make_sample_mapping <- function(pheno, sce){
    df <- data.frame(donor.expt.day = colnames(pheno))
#     print(head(df))
    df$donor = gsub("*-.*","",df$donor.expt.day)
    df$expt = gsub("*-.*","",gsub(".*-e","e",df$donor.expt.day))
    df$day = gsub(".*-","",df$donor.expt.day)
#     print(head(df))
    df1 <- data.frame(donor_long_id = unique(sce$donor_long_id), donor = unique(sce$donor_short_id))
#     print(head(df1))
    df2 <- left_join(df, df1, by = "donor")
#     print(head(df2))
    # genotype_samples phenotype_samples
    samples = df2[,c(5,1)]
#     print(head(samples))
    colnames(samples) = c()
    samples
}

In [ ]:
#### binning by pseudotime
df = data.frame(cells = colnames(sce), pseudo = sce$pseudo)
df = df[order(df$pseudo),]
df$rank = 1:nrow(df)
head(df)

In [73]:
length(seq(1, tot-window, by = step))
tot
seq(1, tot-window, by = step)

tot-window

j = 0
# print (j)
# print(head(seq(1, tot-window, by = step)))
for (ii in seq(1, tot-window, by = step)){
    print(ii)
#     if(ii < 234230){next}
    bin_cells = df[df$rank >= round(ii) & df$rank < round(ii+window),"cells"]
    mysce = sce[,bin_cells]
#     print(ncol(mysce[,mysce$day == "day0"]))
    l = get_combinations(mysce)
    m = get_means_byexpt(mysce, nrow(mysce), l)
    mean_pcs = prcomp(t(m))$x[,1:10]
    smf = make_sample_mapping(m, mysce)
    j = j + 1
    phenotype_filename = paste0(output.dir,"bin",j,".pheno")
    covariate_filename = paste0(output.dir,"bin",j,".covs")
    samplemapping_filename = paste0(output.dir,"bin",j,".samples")
    write.table(m, phenotype_filename, sep = "\t", quote = F, col.names = NA)
    write.table(mean_pcs, covariate_filename, sep = "\t", quote = F, col.names = NA)
    write.table(smf, samplemapping_filename, sep = "\t", quote = F, row.names = F, col.names = F)
}

In [68]:
j = 31
ii = 26132.9
bin_cells = df[df$rank >= round(ii),"cells"]
mysce = sce[,bin_cells]
#     print(ncol(mysce[,mysce$day == "day0"]))
l = get_combinations(mysce)
m = get_means_byexpt(mysce, nrow(mysce), l)
mean_pcs = prcomp(t(m))$x[,1:10]
smf = make_sample_mapping(m, mysce)

phenotype_filename = paste0(output.dir,"bin",j,".pheno")
covariate_filename = paste0(output.dir,"bin",j,".covs")
samplemapping_filename = paste0(output.dir,"bin",j,".samples")
write.table(m, phenotype_filename, sep = "\t", quote = F, col.names = NA)
write.table(mean_pcs, covariate_filename, sep = "\t", quote = F, col.names = NA)
write.table(smf, samplemapping_filename, sep = "\t", quote = F, row.names = F, col.names = F)

In [72]:
head(mean_pcs)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
juuy_2-expt_44-day1,-30.37454,17.70143,-17.3643640,28.767290,-29.57952,12.05118,-5.6040005,1.0037126,-4.4545085,14.867224
dixh_2-expt_21-day2,36.62395,26.98611,0.3278896,-5.800941,-17.44653,22.69369,-3.7671211,-1.5625529,2.5842009,2.433113
fawm_2-expt_21-day2,41.90682,25.00021,-1.0284605,-5.625613,-20.36903,23.11264,-0.4774983,-0.7451078,7.6574624,4.872406
koqx_1-expt_21-day2,42.13907,20.74525,1.1785903,-2.132479,-18.28414,30.30899,2.7609909,-8.2975287,-0.5786015,-1.127607
naju_1-expt_21-day2,53.05615,22.08777,0.7714651,4.667380,-12.60444,25.58052,3.6432695,-2.3548024,-1.3419568,-3.409163
oebj_1-expt_21-day2,36.31176,20.01570,9.4416417,-2.039664,-14.96616,33.84707,4.3934090,-10.7571124,3.3042272,-1.710720


In [ ]:
##### binning by cell cycle (G1S)

In [9]:
coexpr_clusters_filename = "/nfs/leia/research/stegle/dseaton/hipsci/singlecell_endodiff/data/sce_merged_afterqc_filt_allexpts.PCA.cluster_means.tsv"
coexpr_clusters <- read.csv(coexpr_clusters_filename, row.names = 1, sep = "\t", header = T)
head(coexpr_clusters)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,⋯,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59
21843_1#10,-2.96677786,-5.051700,2.557705,1.7846318,-3.8000932,2.7520889,0.04011419,-0.65210263,-2.4088645,-1.4745952,⋯,0.9746980,1.157624,1.0561422,0.6647575,1.0319868,0.9611851,1.1291618,1.1297945,1.105578,1.475525
21843_1#100,2.99582684,-9.416975,3.945243,4.6289860,-1.2671332,1.6378075,0.24622875,2.11823943,-0.1742999,-1.1184431,⋯,0.8779742,1.035389,0.7064237,0.7908379,0.9249539,0.9632030,0.9793299,0.9073227,1.006672,1.317119
21843_1#101,-0.08867775,-6.769028,5.613472,1.8904508,-1.3355565,0.6089684,0.80128325,0.42480083,1.0950508,1.1932872,⋯,0.9319036,1.083866,0.8746851,0.6844323,0.9858303,0.9590457,1.1212404,1.1387067,1.041555,1.390104
21843_1#102,-5.00171746,-4.727163,2.400886,2.0255520,-2.0095244,5.2201763,1.46083794,-1.89020695,-0.4836101,0.6099583,⋯,0.9637354,1.166783,1.0856965,0.5970522,1.0349560,0.8610726,1.1574574,1.1208192,1.088636,1.499318
21843_1#103,-2.08193872,-6.209893,4.675325,0.5933318,-0.6481828,3.4695832,-0.36674697,-0.72786039,-0.6366988,0.3395057,⋯,1.0077623,1.192433,1.0881635,0.6984261,0.9993791,0.9143417,1.0999614,1.0552629,1.053882,1.557355
21843_1#105,-0.71012454,-6.392112,4.809889,0.5048616,-0.4834257,1.2074208,3.08038259,0.05347892,-1.7887607,-2.4269346,⋯,0.9450494,1.060659,1.0799314,0.8152264,0.9843815,0.9029240,1.0742548,1.1586082,1.167950,1.494102


In [10]:
df = data.frame(cells = colnames(sce), G1S = coexpr_clusters$X10)
df = df[order(df$G1S),]
df$rank = 1:nrow(df)
head(df)

,cells,G1S,rank
2648,21554_2#306,0.1190575,1
4593,22606_7#171,0.1255150,2
3581,22194_6#67,0.1366671,3
4910,24327_4#274,0.1432052,4
2531,21554_1#43,0.1457136,5
4572,22606_7#116,0.1613649,6


In [11]:
step.pct = 0.025
window.pct = 0.25
output.dir = "/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_processed/scQTLs/binning_analysis/input_files_cellcycle/"
sce.file = "/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_processed/merged/20180618/sce_merged_afterqc_filt_allexpts.rds"

In [12]:
tot = ncol(sce)
window = window.pct*tot
step = step.pct*tot

In [13]:
length(seq(1, tot-window, by = step))
tot
seq(1, tot-window, by = step)
tot-window

[1] 30

[1] 36044

[1]     1.0   902.1  1803.2  2704.3  3605.4  4506.5  5407.6  6308.7  7209.8
[10]  8110.9  9012.0  9913.1 10814.2 11715.3 12616.4 13517.5 14418.6 15319.7
[19] 16220.8 17121.9 18023.0 18924.1 19825.2 20726.3 21627.4 22528.5 23429.6
[28] 24330.7 25231.8 26132.9

[1] 27033

In [19]:
j = 0
# print (j)
# print(head(seq(1, tot-window, by = step)))
for (ii in seq(1, tot-window, by = step)){
    print(ii)
#     if(ii < 234230){next}
    bin_cells = df[df$rank >= round(ii) & df$rank < round(ii+window),"cells"]
    mysce = sce[,bin_cells]
#     print(ncol(mysce[,mysce$day == "day0"]))
    l = get_combinations(mysce)
    m = get_means_byexpt(mysce, nrow(mysce), l)
    mean_pcs = prcomp(t(m))$x[,1:10]
    smf = make_sample_mapping(m, mysce)
    j = j + 1
    phenotype_filename = paste0(output.dir,"bin",j,".pheno")
    covariate_filename = paste0(output.dir,"bin",j,".covs")
    samplemapping_filename = paste0(output.dir,"bin",j,".samples")
    write.table(m, phenotype_filename, sep = "\t", quote = F, col.names = NA)
    write.table(mean_pcs, covariate_filename, sep = "\t", quote = F, col.names = NA)
    write.table(smf, samplemapping_filename, sep = "\t", quote = F, row.names = F, col.names = F)
}

[1] 1
[1] 902.1
[1] 1803.2
[1] 2704.3
[1] 3605.4
[1] 4506.5
[1] 5407.6
[1] 6308.7
[1] 7209.8
[1] 8110.9
[1] 9012
[1] 9913.1
[1] 10814.2
[1] 11715.3
[1] 12616.4
[1] 13517.5
[1] 14418.6
[1] 15319.7
[1] 16220.8
[1] 17121.9
[1] 18023
[1] 18924.1
[1] 19825.2
[1] 20726.3
[1] 21627.4
[1] 22528.5
[1] 23429.6
[1] 24330.7
[1] 25231.8
[1] 26132.9


In [20]:
j = 31
ii = 26132.9
bin_cells = df[df$rank >= round(ii),"cells"]
mysce = sce[,bin_cells]
#     print(ncol(mysce[,mysce$day == "day0"]))
l = get_combinations(mysce)
m = get_means_byexpt(mysce, nrow(mysce), l)
mean_pcs = prcomp(t(m))$x[,1:10]
smf = make_sample_mapping(m, mysce)

phenotype_filename = paste0(output.dir,"bin",j,".pheno")
covariate_filename = paste0(output.dir,"bin",j,".covs")
samplemapping_filename = paste0(output.dir,"bin",j,".samples")
write.table(m, phenotype_filename, sep = "\t", quote = F, col.names = NA)
write.table(mean_pcs, covariate_filename, sep = "\t", quote = F, col.names = NA)
write.table(smf, samplemapping_filename, sep = "\t", quote = F, row.names = F, col.names = F)

In [ ]:
## respiration

In [27]:
step.pct = 0.025
window.pct = 0.25
output.dir = "/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_processed/scQTLs/binning_analysis/input_files_respiration/"
sce.file = "/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_processed/merged/20180618/sce_merged_afterqc_filt_allexpts.rds"

In [31]:
df = data.frame(cells = colnames(sce), Respiration = coexpr_clusters$X0)
df = df[order(df$Respiration),]
df$rank = 1:nrow(df)
head(df)

,cells,Respiration,rank
2192,21554_6#44,0.1522569,1
2009,21554_5#62,0.1851594,2
2007,21554_5#5,0.1925315,3
2719,21554_3#116,0.2236311,4
2010,21554_5#69,0.2386049,5
1879,21554_5#139,0.2388660,6


In [ ]:
j = 0
# print (j)
# print(head(seq(1, tot-window, by = step)))
for (ii in seq(1, tot-window, by = step)){
    print(ii)
#     if(ii < 234230){next}
    bin_cells = df[df$rank >= round(ii) & df$rank < round(ii+window),"cells"]
    mysce = sce[,bin_cells]
#     print(ncol(mysce[,mysce$day == "day0"]))
    l = get_combinations(mysce)
    m = get_means_byexpt(mysce, nrow(mysce), l)
    mean_pcs = prcomp(t(m))$x[,1:10]
    smf = make_sample_mapping(m, mysce)
    j = j + 1
    phenotype_filename = paste0(output.dir,"bin",j,".pheno")
    covariate_filename = paste0(output.dir,"bin",j,".covs")
    samplemapping_filename = paste0(output.dir,"bin",j,".samples")
    write.table(m, phenotype_filename, sep = "\t", quote = F, col.names = NA)
    write.table(mean_pcs, covariate_filename, sep = "\t", quote = F, col.names = NA)
    write.table(smf, samplemapping_filename, sep = "\t", quote = F, row.names = F, col.names = F)
}

In [ ]:
j = 31
ii = 26132.9
bin_cells = df[df$rank >= round(ii),"cells"]
mysce = sce[,bin_cells]
#     print(ncol(mysce[,mysce$day == "day0"]))
l = get_combinations(mysce)
m = get_means_byexpt(mysce, nrow(mysce), l)
mean_pcs = prcomp(t(m))$x[,1:10]
smf = make_sample_mapping(m, mysce)

phenotype_filename = paste0(output.dir,"bin",j,".pheno")
covariate_filename = paste0(output.dir,"bin",j,".covs")
samplemapping_filename = paste0(output.dir,"bin",j,".samples")
write.table(m, phenotype_filename, sep = "\t", quote = F, col.names = NA)
write.table(mean_pcs, covariate_filename, sep = "\t", quote = F, col.names = NA)
write.table(smf, samplemapping_filename, sep = "\t", quote = F, row.names = F, col.names = F)